<a href="https://colab.research.google.com/github/Sushila-Phogat/Data-science-with-Python/blob/master/PreInterviewTaskfinal.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [67]:
#importing necessary packages and data from my github account
import pandas as pd
import numpy as np
df=pd.read_csv('https://raw.githubusercontent.com/Sushila-Phogat/colabtools/master/PreInterviewTaskData.csv')
data=df
data=pd.DataFrame(data)
print(data.head())

  account        date  fraud merchant
0      A0  01/01/2015  False      M98
1      A0  01/01/2015  False      M50
2      A0  01/01/2015  False      M55
3      A0  01/01/2015  False      M14
4      A0  01/01/2015  False      M10


In [73]:
#Describing main deatils of dataset
data.describe()

,account,date,fraud,merchant
count,89851,89851,89851,89851
unique,1000,30,2,100
top,A831,12/01/2015,False,M8
freq,121,3081,88503,998


In [75]:
#Describing main deatils of dataset for Defrauded transaction
data[(data['fraud'].astype(str).str.strip()=='False')].describe()

,account,date,fraud,merchant
count,88503,88503,88503,88503
unique,1000,30,1,100
top,A831,01/01/2015,False,M8
freq,119,3051,88503,985


In [76]:
#Describing main deatils of dataset for frauded transaction
data[(data['fraud'].astype(str).str.strip()=='True')].describe()

,account,date,fraud,merchant
count,1348,1348,1348,1348
unique,617,30,1,100
top,A425,22/01/2015,True,M57
freq,16,188,1348,22


In [0]:
# Total number of accounts which are exploited
len(set(data[(data['fraud'].astype(str).str.strip()=='True')]['account']).intersection(set(data[(data['fraud'].astype(str).str.strip()=='True')]['account'])))

617

In [0]:
# Function to Detection Outlier on one-dimentional datasets.
def find_anomalies(random_data,st):
    anomalies = []
    # Set upper and lower limit to st standard deviation
    random_data_std = np.std(random_data)
    random_data_mean = np.mean(random_data)
    anomaly_cut_off = random_data_std * st
    upper_limit = random_data_mean + anomaly_cut_off
    
    # Generate outliers
    for outlier in random_data:
        if outlier > upper_limit:
            anomalies.append(outlier)
            anomalies.append(outlier-upper_limit)
    return anomalies


In [84]:
#find exploitation dates with counts and accounts on base of result of anomalies detection function
expltalldict={}
exploitdt={}
expltacnt=[]
exploitrst=[]
inp=find_anomalies(data[(data['fraud'].astype(str).str.strip()=='True')].groupby(['date']).count().iloc[:,-1],2.5) #datewise counts of all frauded transactions
exploitrst.extend(inp) #Exploitation result of frauded transactions count
for i in data[(data['fraud'].astype(str).str.strip()=='True')]['date'].unique():
  for j in data[(data['fraud'].astype(str).str.strip()=='True') & (data['date']==i)].groupby(['date']).count().iloc[:,-1]:
    expltalldict.update({i:j}) #all dates and its counts of all frauded transactions

#Creating dictionary with abnormal dates and its frauded transactions count 
for i in range(len(exploitrst)):
    for (key, value) in expltalldict.items():
      if str(value) == str(exploitrst[i]):
        exploitdt.update({str(key):str(value)}) 
        expltacnt.extend(data[(data['fraud'].astype(str).str.strip()=='True') & (data['date']==key)]['account'].unique()) #List of all exploited accounts
print('Exploitation dates with counts of Transaction on base of abnormality',exploitdt)
print('Unique accounts which exploited during above mentioned exploition dates',set(expltacnt))


Exploitation dates with counts of Transaction on base of abnormality {'21/01/2015': '184', '22/01/2015': '188', '23/01/2015': '179'}
Unique accounts which exploited during above mentioned exploition dates {'A238', 'A355', 'A817', 'A493', 'A581', 'A90', 'A284', 'A183', 'A896', 'A283', 'A556', 'A203', 'A270', 'A461', 'A882', 'A6', 'A657', 'A874', 'A541', 'A746', 'A973', 'A92', 'A951', 'A665', 'A69', 'A167', 'A381', 'A699', 'A468', 'A291', 'A485', 'A723', 'A148', 'A952', 'A610', 'A998', 'A330', 'A836', 'A824', 'A444', 'A655', 'A825', 'A379', 'A603', 'A57', 'A517', 'A841', 'A987', 'A781', 'A991', 'A166', 'A325', 'A122', 'A119', 'A385', 'A634', 'A560', 'A434', 'A318', 'A437', 'A8', 'A786', 'A682', 'A697', 'A435', 'A425', 'A942', 'A907', 'A921', 'A40', 'A490', 'A645', 'A943', 'A494', 'A551', 'A809', 'A145', 'A701', 'A930', 'A743', 'A726', 'A430', 'A727', 'A349', 'A802', 'A324', 'A615', 'A372', 'A837', 'A480', 'A16', 'A834', 'A403', 'A771', 'A731', 'A321', 'A111', 'A110', 'A397', 'A838', 'A70

In [7]:
#Normal anlaysis without matching exploited accounts for abnormal defrauded transactions and creating anomlymer0 dictionary containing suspected compromised merchants with abnormality result
dictdt={}
anomlymer0={}
for i in data[(data['fraud'].astype(str).str.strip()=='False')]['merchant'].unique():
  inp=data[(data['fraud'].astype(str).str.strip()=='False') & (data['merchant'].astype(str).str.strip()==i)].groupby(['merchant','date']).count().iloc[:,-1]
  result=find_anomalies(inp,2.5) #Check abnormality with anomaly_cut_off is 2.5 times of standard deviation
  if len(result)!=0:
    listOfdat=[]
    for j in data[(data['fraud'].astype(str).str.strip()=='False') & (data['merchant'].astype(str).str.strip()==i)]['date'].unique():
      for k in data[(data['fraud'].astype(str).str.strip()=='False') & (data['merchant'].astype(str).str.strip()==i) & (data['date'].astype(str).str.strip()==j)].groupby(['date']).count().iloc[:,-1]:
        dictdt.update({str(j):str(k)})
    for (key, value) in dictdt.items():
      if str(value) == str(result[0]):
        listOfdat = key
        anomlymer0.update({str(i):str(key)})
    print(i,listOfdat,result)


M98 12/01/2015 [42, 0.14938628074362015]
M50 04/01/2015 [44, 3.5015247715286506]
M14 28/01/2015 [45, 2.8772415372337434]
M10 30/01/2015 [49, 3.543920565419654]
M33 01/01/2015 [42, 1.9578696975417529]
M26 15/01/2015 [45, 2.6509109647715334]
M21 08/01/2015 [49, 3.2617184147262606, 46, 0.26171841472626056]
M70 01/01/2015 [48, 3.6792698265290653]
M69 14/01/2015 [49, 1.9607815473821546]
M46 05/01/2015 [44, 1.0506869942160506]
M12 24/01/2015 [44, 0.13020410527151682]
M30 23/01/2015 [46, 2.7135563340304714]
M41 04/01/2015 [44, 0.39702412580335533]
M28 09/01/2015 [42, 1.1380962103093992]
M93 01/01/2015 [43, 0.05891682380070051]
M24 01/01/2015 [44, 1.0237084647307668]
M76 15/01/2015 [43, 0.8322511278810367]
M3 09/01/2015 [47, 0.16071777722640945]
M45 01/01/2015 [42, 0.8917349728611654]
M81 10/01/2015 [42, 0.36853645925264544]
M75 08/01/2015 [42, 0.19895837402025762]
M34 12/01/2015 [48, 4.012772858428619]
M61 19/01/2015 [46, 2.354021250743891, 45, 1.354021250743891]


In [119]:
#Normal anlaysis with matching exploited accounts for abnormal defrauded transactions and creating anomlymer dictionary containing suspected compromised merchants with abnormality result
anomlymer={}
for j in data[(data['fraud'].astype(str).str.strip()=='False') ]['merchant'].unique():
  inp=[]
  dictdt={}
  for i in data[(data['fraud'].astype(str).str.strip()=='False')]['date'].unique():
    inp1=len(list(set(expltacnt).intersection(set(data[(data['fraud'].astype(str).str.strip()=='False') & (data['merchant'].astype(str).str.strip()==j) & (data['date'].astype(str).str.strip()==i)]['account'].unique()))))
    inp.append(inp1)
  result=find_anomalies(inp,1)  #Check abnormality with anomaly_cut_off is 1 times of standard deviation
  if len(result)!=0:
    listOfdat=[]
    for i in data[(data['fraud'].astype(str).str.strip()=='False')]['date'].unique():
      inp=len(set(expltacnt).intersection(set(data[(data['fraud'].astype(str).str.strip()=='False') & (data['merchant'].astype(str).str.strip()==j) & (data['date'].astype(str).str.strip()==i)]['account'].unique())))
      dictdt.update({str(i):str(inp)})
    for (key, value) in dictdt.items():
      if str(value) == str(result[0]):
        listOfdat = key
        anomlymer.update({str(j):str(key)})
  print(j,listOfdat,result)



M98 01/01/2015 [8, 2.272967209798548, 9, 3.272967209798548, 6, 0.2729672097985478, 6, 0.2729672097985478]
M50 29/01/2015 [7, 0.8810735390189421, 8, 1.8810735390189421, 9, 2.881073539018942, 7, 0.8810735390189421, 7, 0.8810735390189421]
M55 20/01/2015 [6, 0.3805039497961422, 7, 1.3805039497961422, 7, 1.3805039497961422, 6, 0.3805039497961422, 6, 0.3805039497961422, 6, 0.3805039497961422]
M14 03/01/2015 [8, 2.140753851237399, 7, 1.140753851237399, 6, 0.14075385123739892, 10, 4.140753851237399]
M10 02/01/2015 [8, 1.8454681465103784, 7, 0.8454681465103784, 7, 0.8454681465103784]
M33 03/01/2015 [7, 1.4553995067804273, 7, 1.4553995067804273, 8, 2.4553995067804273]
M82 26/01/2015 [7, 0.6159334623545885, 10, 3.6159334623545885, 7, 0.6159334623545885, 7, 0.6159334623545885, 7, 0.6159334623545885]
M42 13/01/2015 [8, 1.5756212963413887, 7, 0.5756212963413887, 8, 1.5756212963413887, 9, 2.5756212963413887, 7, 0.5756212963413887]
M22 06/01/2015 [12, 5.32605408559969, 7, 0.32605408559969007, 7, 0.326

In [120]:
#Detection of Compromised merchant with all dates of information harvestion

#All abnormal defrauded transactions which use same account which exploited later and dates with all accounts count of which information harvested  
anmldtl=[]
for key,value in anomlymer.items():
  anmlymerdact=[]
  for i in data['date'].unique():
    if i<min(exploitdt.keys()):
      anmlymerdact.append(len(set(expltacnt).intersection(set(data[(data['fraud'].astype(str).str.strip()=='False') & (data['date']==i)  & (data['merchant']==key)]['account'].unique()))))
  if len(find_anomalies(anmlymerdact,1))!=0:
    anmldtl.append(key)
    anmldtl.append(find_anomalies(anmlymerdact,1))
    
frdsum=[]
frdmer={}

# Loop for suming up all counts of transaction in abnormal list
for i in range(0,len(anmldtl),2):
  mer=anmldtl[i]
  frdacsum=0
  y=[]
  for j in range(0,len(anmldtl[i+1]),2):
    frdacsum=frdacsum+anmldtl[i+1][j] #sum of all frauded accounts transations count which are in abnormarl list for particular merchant 
    y.append(anmldtl[i+1][j])
  frdsum.append(frdacsum)
  frdmer.update({mer:y})

#finding out merchant with maximum abnormal transaction with defrauded indicator and with frauded accounts
for key, value in frdmer.items():
  if sum(value)==max(frdsum):
    cmpmer=key
print('Compromised merchnat ',cmpmer)

#Dates of Compromise for that merchant
fradt={}
for k in data['date'].unique():
  if k<min(exploitdt.keys()):
    fradt.update({k:len(set(expltacnt).intersection(set(data[(data['fraud'].astype(str).str.strip()=='False') & (data['date']==k)  & (data['merchant']==cmpmer)]['account'].unique())))})

dts={}
for key,value in fradt.items():
  for l in list(dict(filter(lambda elem: elem[0] ==cmpmer , frdmer.items())).values())[0]:
    if l==value:
      dts.update({key:l})

print('Dates of compromise', list(dts.keys()))

Compromised merchnat  M18
Dates of compromise ['15/01/2015', '16/01/2015']


In [121]:
#More insight for abnormal defrauded transactions which use same account which exploited later and date with maximum accounts count of which information harvested  

for key,value in anomlymer.items():
  anmact=[]
  anmact.extend(data[(data['fraud'].astype(str).str.strip()=='False') & (data['merchant']==key) & (data['date']==value)]['account'].unique())
  print(key,value,len(set(expltacnt).intersection(set(anmact))),set(expltacnt).intersection(set(anmact)))

M98 01/01/2015 8 {'A841', 'A215', 'A37', 'A727', 'A182', 'A183', 'A40', 'A665'}
M50 29/01/2015 7 {'A590', 'A824', 'A336', 'A443', 'A541', 'A825', 'A998'}
M55 20/01/2015 6 {'A490', 'A726', 'A403', 'A110', 'A793', 'A430'}
M14 03/01/2015 8 {'A704', 'A699', 'A834', 'A90', 'A731', 'A942', 'A886', 'A381'}
M10 02/01/2015 8 {'A238', 'A16', 'A391', 'A581', 'A145', 'A200', 'A973', 'A166'}
M33 03/01/2015 7 {'A882', 'A560', 'A145', 'A139', 'A973', 'A372', 'A430'}
M82 26/01/2015 7 {'A391', 'A494', 'A874', 'A475', 'A157', 'A318', 'A921'}
M42 13/01/2015 8 {'A461', 'A273', 'A444', 'A551', 'A425', 'A942', 'A111', 'A743'}
M22 06/01/2015 12 {'A493', 'A824', 'A215', 'A634', 'A291', 'A560', 'A727', 'A701', 'A157', 'A541', 'A743', 'A203'}
M44 06/01/2015 7 {'A392', 'A291', 'A702', 'A809', 'A379', 'A8', 'A726'}
M52 20/01/2015 7 {'A720', 'A645', 'A443', 'A324', 'A615', 'A92', 'A166'}
M26 27/01/2015 7 {'A122', 'A250', 'A210', 'A921', 'A907', 'A397', 'A469'}
M59 07/01/2015 8 {'A336', 'A560', 'A90', 'A349', 'A728

In [124]:
for i in data['date'].unique():
  if i<min(exploitdt.keys()):
    print(i,len(set(expltacnt).intersection(set(data[(data['fraud'].astype(str).str.strip()=='False') & (data['date']==i)  & (data['merchant']=='M18')]['account'].unique()))))

01/01/2015 2
02/01/2015 4
03/01/2015 1
04/01/2015 4
05/01/2015 5
06/01/2015 2
07/01/2015 3
08/01/2015 5
09/01/2015 3
10/01/2015 3
11/01/2015 3
12/01/2015 5
13/01/2015 6
14/01/2015 4
15/01/2015 21
16/01/2015 32
17/01/2015 1
18/01/2015 0
19/01/2015 3
20/01/2015 6
